# Online Retail RFM and Analysis

Using the Online Retail dataset from the UCI Machine Learning Repository for exploratory data analysis and RFM Analysis.

This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail. The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

#### Feature Information:

**InvoiceNo**: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.
<br>
**StockCode**: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.
<br>
**Description**: Product (item) name. Nominal.
<br>
**Quantity**: The quantities of each product (item) per transaction. Numeric.
<br>
**InvoiceDate**: Invoice Date and time. Numeric, the day and time when each transaction was generated.
<br>
**UnitPrice**: Unit price. Numeric, Product price per unit in sterling.
<br>
**CustomerID**: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.
<br>
**Country**: Country name. Nominal, the name of the country where each customer resides. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("OnlineRetail.csv")
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom


### 1. Data Exploration

In [3]:
df.duplicated().value_counts()

False    536641
True       5268
dtype: int64

In [4]:
df = df.drop_duplicates()

In [5]:
df.shape

(536641, 8)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 536641 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    536641 non-null  object 
 1   StockCode    536641 non-null  object 
 2   Description  535187 non-null  object 
 3   Quantity     536641 non-null  int64  
 4   InvoiceDate  536641 non-null  object 
 5   UnitPrice    536641 non-null  float64
 6   CustomerID   401604 non-null  float64
 7   Country      536641 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 36.8+ MB


In [7]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135037
Country             0
dtype: int64

In [8]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,536641.000000,536641.000000,401604.000000
mean,9.620029,4.632656,15281.160818
std,219.130156,97.233118,1714.006089
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13939.000000
50%,3.000000,2.080000,15145.000000
75%,10.000000,4.130000,16784.000000
max,80995.000000,38970.000000,18287.000000


There are negative values in the Quantity and UnitPrice columns. These are possibly canceled and returned orders.

In [9]:
df[df.Quantity < 0].shape[0]

10587

In [10]:
df[df.Quantity < 0].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,01-12-2010 09:41,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,01-12-2010 09:49,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,01-12-2010 10:24,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,01-12-2010 10:24,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,01-12-2010 10:24,0.29,17548.0,United Kingdom


In [11]:
df[df.UnitPrice < 0].shape[0]

2

It is not usable for our analysis we need to drop them.

In [12]:
df[df.UnitPrice < 0].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299983,A563186,B,Adjust bad debt,1,12-08-2011 14:51,-11062.06,NaN,United Kingdom
299984,A563187,B,Adjust bad debt,1,12-08-2011 14:52,-11062.06,NaN,United Kingdom


If the invoice number starts with the letter "C", it means that the order was cancelled, i learned this from other people notebooks.

In [13]:
df['cancellation'] = df.InvoiceNo.str.extract('([C])').fillna(0).replace({'C':1})
df.cancellation.value_counts()

0    527390
1      9251
Name: cancellation, dtype: int64

In [14]:
df[df.cancellation == 1]['CustomerID'].nunique() / df.CustomerID.nunique() * 100

36.34492223238792

36% of the orders were cancelled.

### 2. Missing Values

In [15]:
df[df.cancellation == 1]['CustomerID'].value_counts(dropna = False).head(5)

NaN        379
14911.0    226
17841.0    136
17511.0    113
15311.0    112
Name: CustomerID, dtype: int64

In [16]:
df = df[df.CustomerID.notnull()]

In [17]:
df = df[(df.Quantity > 0) & (df.UnitPrice > 0)]

In [18]:
df[df.cancellation == 1]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,cancellation


In [19]:
df = df.drop('cancellation', axis = 1)

### 3. EDA

unique number of InvoiceNo per customer

In [20]:
df.groupby('CustomerID')['InvoiceNo'].nunique().sort_values(ascending = False)

CustomerID
12748.0    209
14911.0    201
17841.0    124
13089.0     97
14606.0     93
          ... 
15314.0      1
15313.0      1
15308.0      1
15307.0      1
15300.0      1
Name: InvoiceNo, Length: 4338, dtype: int64

In [21]:
# Average number of unique products for each customer in each order.
mean_of_unique_items= round(df.groupby(['CustomerID',
                      'InvoiceNo']).agg({'StockCode':lambda x:x.nunique()}).groupby('CustomerID')['StockCode'].mean(),
                      1).sort_values(ascending=False)
mean_of_unique_items

CustomerID
14096.0    298.8
12378.0    219.0
16984.0    191.0
12688.0    171.0
15464.0    155.0
           ...  
17923.0      1.0
17925.0      1.0
15753.0      1.0
17948.0      1.0
12346.0      1.0
Name: StockCode, Length: 4338, dtype: float64

Let's see how this compares to the number of unique products per customer.

In [22]:
# The number of unique products purchased by each customer.
num_of_unique_product= pd.DataFrame(df.groupby('CustomerID').StockCode.nunique()).rename(columns={'StockCode':'num_of_unique_product'})

# The number of orders for each customer.
num_of_order = df.groupby('CustomerID').InvoiceNo.nunique()

In [23]:
pd.concat([mean_of_unique_items,
           num_of_order,
           num_of_unique_product],
           axis=1).rename(columns={'StockCode': "mean_of_unique_items",
                                   'InvoiceNo': 'num_of_order'}).sort_values('num_of_order', ascending=False)

,mean_of_unique_items,num_of_order,num_of_unique_product
CustomerID,,,
12748.0,20.5,209,1768
14911.0,28.2,201,1787
17841.0,60.3,124,1323
13089.0,18.7,97,636
14606.0,28.6,93,819
...,...,...,...
16337.0,10.0,1,10
17871.0,10.0,1,10
17887.0,10.0,1,10


In [24]:
df["TotalPrice"] = df.UnitPrice * df.Quantity

In [25]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom,20.34


In [26]:
df2 = pd.DataFrame(df.groupby('Country').TotalPrice.sum().apply(lambda x: round(x, 2))).sort_values('TotalPrice', ascending = False)
df2.head()

,TotalPrice
Country,
United Kingdom,7285024.64
Netherlands,285446.34
EIRE,265262.46
Germany,228678.40
France,208934.31


In [27]:
df2['perc_of_TotalPrice'] = round(df2.TotalPrice / df2.TotalPrice.sum() * 100, 2)
df2

,TotalPrice,perc_of_TotalPrice
Country,,
United Kingdom,7285024.64,81.97
Netherlands,285446.34,3.21
EIRE,265262.46,2.98
Germany,228678.40,2.57
France,208934.31,2.35
Australia,138453.81,1.56
Spain,61558.56,0.69
Switzerland,56443.95,0.64
Belgium,41196.34,0.46


#### 2. RFM Analysis
**RECENCY (R):** Time since last purchase
<br>
**FREQUENCY (F):** Total number of purchases
<br>
**MONETARY VALUE (M):** Total monetary value

In [28]:
from datetime import datetime as dt
import warnings
warnings.filterwarnings('ignore')

In [29]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

df_uk=df[df.Country=='United Kingdom']
df_uk.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-01-12 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-01-12 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom,20.34


In [30]:
# a point of reference date 
ref_date = max(df['InvoiceDate'])
ref_date

Timestamp('2011-12-10 17:19:00')

In [31]:
df_uk['Date']=df_uk['InvoiceDate'].apply(lambda x: x.date())
df_uk.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,Date
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-01-12 08:26:00,2.55,17850.0,United Kingdom,15.30,2010-01-12
1,536365,71053,WHITE METAL LANTERN,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-01-12
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-01-12 08:26:00,2.75,17850.0,United Kingdom,22.00,2010-01-12


In [32]:
df_uk['Last_Purchase_Date'] = df_uk.groupby(['CustomerID'])['Date'].transform(max)
df_uk.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,Date,Last_Purchase_Date
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-01-12 08:26:00,2.55,17850.0,United Kingdom,15.30,2010-01-12,2010-02-12
1,536365,71053,WHITE METAL LANTERN,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-01-12,2010-02-12
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-01-12 08:26:00,2.75,17850.0,United Kingdom,22.00,2010-01-12,2010-02-12
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-01-12,2010-02-12
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-01-12,2010-02-12


#### 3.1 Recency

In [ ]:
df_uk['Recency'] = df_uk.groupby('CustomerID')['Last_Purchase_Date'].apply(lambda x:ref_date.date() - x)
df_uk['Recency'] = df_uk.agg({'Recency':lambda x:x.astype('timedelta64[D]')})
df_uk.head()

In [ ]:
df_uk = df_uk.drop('Last_Purchase_Date', axis=1)

In [ ]:
plt.subplots(figsize=(15, 5))
sns.distplot(df_uk.groupby('CustomerID')['Recency'].max(), kde=False, bins=80)
plt.title('Recency Value Distribution', fontsize = 15)
plt.xlabel('Recency')
plt.ylabel('Count');

#### 3.2 Frequency

In [ ]:
df_uk['Frequency'] = df_uk.groupby('CustomerID').InvoiceNo.transform('nunique')

In [ ]:
plt.figure(figsize=(15, 5))
sns.distplot(df_uk.groupby('CustomerID')['Frequency'].max(), kde=False, bins=200)
plt.title('Frequency Value Distribution', fontsize = 15)
plt.xlim(-10, 60)
plt.xlabel('Frequency')
plt.ylabel('Count');

#### 3.3 Monetary

In [ ]:
df_uk['Monetary'] = df_uk.groupby('CustomerID').TotalPrice.transform('sum')

In [ ]:
plt.subplots(figsize=(15, 5))
sns.distplot(df_uk.groupby('CustomerID')['Monetary'].max(), kde=False, bins=400)
plt.title('Monetary Value Distribution', fontsize = 15)
plt.xlim(-10000, 40000)
plt.xlabel('Monetary')
plt.ylabel('Count')

#### 3.4 RFM Table

In [ ]:
df_rfm = df_uk[['Recency','Frequency','Monetary']].drop_duplicates().rename(index=df_uk['CustomerID'])
df_rfm.sort_index()

4. Customer Segmentation with RFM Scores

RFM Scoring

In [ ]:
#  Converting Recency, Frequency and Monetary metrics to scores between 1-5
#  with the help of qcut and recording these scores as recency_score, frequency_score and monetary_score
df_rfm["RecencyScore"] = pd.qcut(x = df_rfm["Recency"], q = 5, labels = ["5", "4", "3", "2", "1"])
df_rfm["FrequencyScore"] = pd.qcut(x = df_rfm["Frequency"].rank(method = "first"), q = 5, labels=["1", "2", "3", "4", "5"])
df_rfm["MonetaryScore"] = pd.qcut(x = df_rfm["Monetary"], q = 5, labels = ["1", "2", "3", "4", "5"])
df_rfm

In [ ]:
# Expressing recency_score and frequency_score as a single variable and saving it as RF_SCORE
df_rfm["RFM_SCORE"] = df_rfm["RecencyScore"].astype(str) + df_rfm["FrequencyScore"].astype(str)
df_rfm

In [ ]:
df_rfm['Segment'] = df_rfm['RFM_SCORE']
df_rfm.head()

In [ ]:
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

In [ ]:
df_rfm.Segment= df_rfm.Segment.replace(seg_map, regex=True)
df_rfm.reset_index(inplace=True)
df_rfm.head()

In [ ]:
df_rfm.groupby('Segment').agg({"index":"count"}).sort_values("index", ascending = False)

In [ ]:
colors  = ("darkorange", "darkseagreen", "orange", "cyan", "cadetblue", "hotpink", "lightsteelblue", "coral",  "mediumaquamarine", "palegoldenrod")
explodes = [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25]

df_rfm["Segment"].value_counts(sort = False).plot.pie(colors = colors,
                                                 textprops = {'fontsize': 12}, 
                                                 autopct = '%4.1f',
                                                 startangle = 90, 
                                                 radius = 2, 
                                                 rotatelabels = True,
                                                 shadow = True, 
                                                 explode = explodes)
plt.ylabel("");